2-3. 하나의 네이버 웹툰과 여러개의 회차에 대한 Image 다운로드 하기 

In [1]:
import requests
from bs4 import BeautifulSoup
import os
import urllib.parse

def extract_title_id_from_url(episode_url):
    """URL에서 titleId 값만 추출"""
    parsed_url = urllib.parse.urlparse(episode_url)
    query_params = urllib.parse.parse_qs(parsed_url.query)
    return query_params.get('titleId', [None])[0]

def download_one_episode(title, no, url):
    """한 회차 상세 페이지에서 이미지들을 다운로드"""
    imgdir_name = os.path.join('img', title, str(no))
    req_header = {'referer': url, 'User-Agent': 'Mozilla/5.0'}

    if os.path.exists(imgdir_name) and os.listdir(imgdir_name):
        print(f"{imgdir_name} 폴더가 이미 존재하고 이미지가 있습니다. 다운로드 생략")
        return

    os.makedirs(imgdir_name, exist_ok=True)

    res = requests.get(url, headers=req_header)
    if not res.ok:
        print(f'Error Code = {res.status_code} ({url})')
        return

    soup = BeautifulSoup(res.text, 'html.parser')
    # 상세 회차 페이지 내 IMAG01을 포함하는 이미지 src 리스트 추출
    img_url_list = [img_tag['src'] for img_tag in soup.select("img[src*='IMAG01']")]

    if not img_url_list:
        print(f"이미지 URL을 찾지 못했습니다: {url}")
        return

    for img_url in img_url_list:
        try:
            img_res = requests.get(img_url, headers=req_header)
            if img_res.ok:
                img_name = os.path.basename(img_url.split("?")[0])
                file_path = os.path.join(imgdir_name, img_name)
                with open(file_path, 'wb') as f:
                    f.write(img_res.content)
                print(f"다운로드 완료: {file_path}")
            else:
                print(f"이미지 다운로드 실패 코드: {img_res.status_code} - {img_url}")
        except Exception as e:
            print(f"이미지 다운로드 중 에러: {e} - {img_url}")

    print(f"'{title}' {no}회차 다운로드 완료: {imgdir_name}")

def get_episode_numbers(title_id, max_pages=1):
    """API에서 회차 번호 리스트만 가져오기"""
    episode_numbers = []
    for page in range(1, max_pages + 1):
        url = f"https://comic.naver.com/api/article/list?titleId={title_id}&tab=week&sort=DESC&page={page}"
        headers = {"User-Agent": "Mozilla/5.0"}
        res = requests.get(url, headers=headers)
        if not res.ok:
            print(f"API 호출 실패: page {page}")
            continue
        data = res.json()
        articles = data.get("articleList", [])
        for art in articles:
            episode_numbers.append(art["no"])
    return episode_numbers

def download_all_episode(title, episode_url):
    title_id = extract_title_id_from_url(episode_url)
    if not title_id:
        print("titleId 추출 실패")
        return

    episode_numbers = get_episode_numbers(title_id,1)

    for no in episode_numbers:
        episode_detail_url = f"https://comic.naver.com/webtoon/detail?titleId={title_id}&no={no}&week=thu"
        download_one_episode(title, no, episode_detail_url)

# 실행 예시
download_all_episode('배달왕', 'https://comic.naver.com/webtoon/list?titleId=823933')


img\배달왕\130 폴더가 이미 존재하고 이미지가 있습니다. 다운로드 생략
img\배달왕\129 폴더가 이미 존재하고 이미지가 있습니다. 다운로드 생략
img\배달왕\128 폴더가 이미 존재하고 이미지가 있습니다. 다운로드 생략
img\배달왕\127 폴더가 이미 존재하고 이미지가 있습니다. 다운로드 생략
img\배달왕\126 폴더가 이미 존재하고 이미지가 있습니다. 다운로드 생략
img\배달왕\125 폴더가 이미 존재하고 이미지가 있습니다. 다운로드 생략
img\배달왕\124 폴더가 이미 존재하고 이미지가 있습니다. 다운로드 생략
img\배달왕\123 폴더가 이미 존재하고 이미지가 있습니다. 다운로드 생략
img\배달왕\122 폴더가 이미 존재하고 이미지가 있습니다. 다운로드 생략
img\배달왕\121 폴더가 이미 존재하고 이미지가 있습니다. 다운로드 생략
img\배달왕\120 폴더가 이미 존재하고 이미지가 있습니다. 다운로드 생략
img\배달왕\119 폴더가 이미 존재하고 이미지가 있습니다. 다운로드 생략
img\배달왕\118 폴더가 이미 존재하고 이미지가 있습니다. 다운로드 생략
img\배달왕\117 폴더가 이미 존재하고 이미지가 있습니다. 다운로드 생략
img\배달왕\116 폴더가 이미 존재하고 이미지가 있습니다. 다운로드 생략
img\배달왕\115 폴더가 이미 존재하고 이미지가 있습니다. 다운로드 생략
img\배달왕\114 폴더가 이미 존재하고 이미지가 있습니다. 다운로드 생략
img\배달왕\113 폴더가 이미 존재하고 이미지가 있습니다. 다운로드 생략
img\배달왕\112 폴더가 이미 존재하고 이미지가 있습니다. 다운로드 생략
img\배달왕\111 폴더가 이미 존재하고 이미지가 있습니다. 다운로드 생략
